### Test functions for DRP_nb module

In [1]:
from importlib import reload
import DRP_nb

In [4]:
import DRP_nb.data_imports
#from DRP_nb import data_imports

In [2]:
reload(DRP_nb)

<module 'DRP_nb' from '/data/home/wpw035/Codebase/DRP_nb/__init__.py'>

In [6]:
prot, rna, ic50_df1, one_hot_cls, one_hot_drugs = DRP_nb.data_imports.read_inputs_phos_ect()
#prot, rna, ic50_df1, one_hot_cls, one_hot_drugs = data_imports.read_inputs_rna_prot()

/data/home/wpw035/.conda/envs/tfGPUforge/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


% of missing prot values 0.386335609896865
num non overlapping prot and target cls: 10
num non overlapping cls: 930


ValueError: too many values to unpack (expected 5)